In [24]:
import pandas as pd
from surprise.model_selection import train_test_split
from surprise import Reader, Dataset  # Lectura y carga del dataset
from surprise import SVD  # Modelo: Singular Value Decomposition
from surprise import accuracy  # Métricas
from sklearn.neighbors import NearestNeighbors
from pickle import load

In [25]:
with open("adult_census_clean.pkl", "rb") as f:
    df = load(f)

In [26]:
df["income"] = df["income"].map({">50K": 1, "<=50K": 0})

In [27]:
df_encoded = pd.get_dummies(df, drop_first=True)

¿Qué se quiere recomendar?

Se quiere recomendar trayectorias socio-laborales que aumenten la probabilidad de que una persona tenga un ingreso anual mayor a 50K USD.

No se recomienda un objeto físico, sino características de personas que lograron mayores ingresos.

¿Quién es el usuario?

El usuario es una persona representada por una fila del dataset.

Cada usuario tiene un perfil socio-laboral:

Edad

Educación

Ocupación

Horas trabajadas

Sexo, raza, país, etc.

¿Qué variables definen el perfil del usuario?



In [28]:
features = [
    "age",
    "education",
    "marital.status",
    "occupation",
    "hours.per.week",
    "sex",
    "native.country"]

X = df[features]
y = df["income"]

Cálculo de similitud entre usuarios

In [30]:
def recomendar_trayectoria(user_profile, model, education_options):
    base_prob = model.predict_proba(pd.DataFrame([user_profile]))[0][1]
    recommendations = []

    for edu in education_options:
        if edu != user_profile["education"]:
            modified_user = user_profile.copy()
            modified_user["education"] = edu
            prob = model.predict_proba(pd.DataFrame([modified_user]))[0][1]

            if prob > base_prob:
                recommendations.append((edu, prob))

    if recommendations:
        best = max(recommendations, key=lambda x: x[1])
        return (
            f"📈 Tu probabilidad actual es {base_prob*100:.1f}%. "
            f"Si alcanzaras el nivel educativo '{best[0]}', "
            f"subiría a {best[1]*100:.1f}%."
        )
    else:
        return f"⚠️ Tu probabilidad actual es {base_prob*100:.1f}%. No se encontraron mejoras simples."

In [ ]:
usuario_test = df[df["income"] == 0].index[0]
df.loc[usuario_test]

age                            82
workclass                 Private
fnlwgt                     132870
education                 HS-grad
education.num                   9
marital.status            Widowed
occupation        Exec-managerial
relationship        Not-in-family
race                        White
sex                        Female
capital.gain                    0
capital.loss                 4356
hours.per.week                 18
native.country      United-States
income                          0
Name: 1, dtype: object

In [ ]:
recomendar_trayectoria(usuario_test, df, X, y)

TypeError: recomendar_trayectoria() takes 3 positional arguments but 4 were given